# Imports and initialization

In [5]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Choose one of the two below cells depending on the environment.

In [6]:
# dir_path = './' #uncomment if in local environment

In [7]:
#uncomment in google colab environment
from google.colab import drive
drive.mount('/content/drive') #path to root directory in Drive
dir_path = './drive/MyDrive/BAKA/' #colab
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
if len(df.columns) > 31:
  df = df.drop(columns=[31])

Mounted at /content/drive


In [8]:
files = os.listdir(dir_path + 'DataCollection')
files

['4', '7', '8', '0', '5', '2', '3', '6', '1']

# Dataset Split
Split samples into Test sets and Training sets <br>
Use 20% of available samples for Testing and the remaining for Training

In [9]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


204   40   164
264   52   212
233   46   187
253   50   203
254   50   204
320   64   256
285   57   228
266   53   213
319   63   256
x_train.shape:  (1923, 60, 31)
y_train.shiape:  (1923,)
x_test.shape:  (475, 60, 31)
y_test.shape:  (475,)


In [10]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)



x_train.shape:  (1923, 60, 31)
y_train.shiape:  (1923,)
x_test.shape:  (475, 60, 31)
y_test.shape:  (475,)


### Scaling and feature labels
Apply min-max scaling technique and relable features corresponding to the description in bachelor thesis's text

In [11]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

Form preprocessed datasets to corresponding shapes as well as shuffle samples

In [12]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [13]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

x_train.shape:  (1923, 60, 31)
y_train.shiape:  (1923,)
x_test.shape:  (475, 60, 31)
y_test.shape:  (475,)


# Model definition

Define Two-layered bidirectional LSTM<br>
Each layer consists of `Bidirectional(LSTM)` cell with addition of `Dropout()` and `BatchNormalization()` to minimize overfitting and decrease learning time

In [14]:
model = Sequential()
model.add(Bidirectional(LSTM(units=60, return_sequences=True ,dtype='float64'),input_shape=x_train.shape[1:],dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Bidirectional(LSTM(units=60 ,dtype='float64') ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Dense(len(files), activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 60, 120)           44160     
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 120)           480       
_________________________________________________________________
dropout (Dropout)            (None, 60, 120)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120)               86880     
_________________________________________________________________
batch_normalization_1 (Batch (None, 120)               480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 9)                 1

# Training

Model training using 150 epochs <br>
Marking checkpoints(models with best `val_accuracy`) to `./Checkpoints` directory

In [15]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=150,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=24,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/150


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


73/73 [==============================] - 10s 41ms/step - loss: 2.9906 - accuracy: 0.1988 - val_loss: 2.0675 - val_accuracy: 0.2332
Epoch 2/150
73/73 [==============================] - 2s 23ms/step - loss: 1.9876 - accuracy: 0.3913 - val_loss: 1.7153 - val_accuracy: 0.4560
Epoch 3/150
73/73 [==============================] - 2s 23ms/step - loss: 1.3615 - accuracy: 0.5624 - val_loss: 1.1860 - val_accuracy: 0.7358
Epoch 4/150
73/73 [==============================] - 2s 23ms/step - loss: 1.0048 - accuracy: 0.6607 - val_loss: 0.7114 - val_accuracy: 0.8601
Epoch 5/150
73/73 [==============================] - 2s 24ms/step - loss: 0.8200 - accuracy: 0.7110 - val_loss: 0.3580 - val_accuracy: 0.9534
Epoch 6/150
73/73 [==============================] - 2s 24ms/step - loss: 0.6271 - accuracy: 0.7936 - val_loss: 0.2110 - val_accuracy: 0.9534
Epoch 7/150
73/73 [==============================] - 2s 24ms/step - loss: 0.5666 - accuracy: 0.8116 - val_loss: 0.1314 - val_accuracy: 0.9741
Epoch 8/150
73/73

In [16]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_bidir', save_format='tf')

Evaluate on test data
20/20 [==============================] - 0s 8ms/step - loss: 0.1914 - accuracy: 0.9579
test loss, test acc: [0.19144324958324432, 0.9578947424888611]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


In [17]:
min_max_scaler.data_min_

array([   0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    , -591.101 ,  -12.7777,
       -190.142 , -626.233 ,    0.    , -238.535 , -648.629 ,    0.    ,
       -251.495 , -643.156 ,    0.    , -243.005 , -675.481 ,    0.    ,
       -220.466 , -607.075 ,    0.    , -158.459 ,    0.    ,    0.    ,
          0.    ])

In [18]:
min_max_scaler.data_max_

array([179.999, 180.   , 180.   , 179.999, 179.991, 179.997, 179.946,
       180.   , 179.985, 179.925, 539.053, 468.363, 720.07 , 581.654,
       522.811, 709.384, 585.01 , 508.522, 705.543, 567.779, 478.013,
       702.861, 544.45 , 478.722, 693.173, 499.893, 466.308, 707.228,
       103.931, 161.082, 127.977])

In [19]:
model.input

<KerasTensor: shape=(None, 60, 31) dtype=float64 (created by layer 'bidirectional_input')>